In [33]:
import torch
import torch.nn as nn
from pyg_pointnet2 import PyGPointNet2NoColor,PyGPointNet2NoColorLoRa
import loralib as lora
import open3d as o3d
import numpy as np
from torch_geometric.data import Data
from torch_geometric.nn import MLP
from torch_geometric.loader import DataLoader
import time
from pc_label_map import color_map


In [30]:
pretrained_path = "checkpoints/pointnet2_s3dis_colorless_seg_x3_45_checkpoint.pth"

# Initialize model
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = PyGPointNet2NoColor(num_classes=13).to(device)

checkpoint = torch.load(pretrained_path, map_location=device)
# Extract the model state dictionary
model_state_dict = checkpoint['model_state_dict']

model.load_state_dict(model_state_dict, strict=False)


<All keys matched successfully>

In [31]:
def apply_lora(module, r=8, alpha=16, verbose=False):
    """
    Recursively replaces Linear layers with LoRA-enabled layers.
    Args:
        module: The module or model to modify.
        r: LoRA rank.
        alpha: LoRA scaling factor.
        verbose: Print layer names during replacement.
    """
    for name, child in module.named_children():
        if isinstance(child, nn.Linear):
            # Replace Linear with LoRA Linear
            lora_layer = lora.Linear(
                in_features=child.in_features,
                out_features=child.out_features,
                r=r,
                lora_alpha=alpha
            )
            
            # Copy original weights and biases
            lora_layer.weight.data = child.weight.data.clone()
            if child.bias is not None:
                lora_layer.bias.data = child.bias.data.clone()
            
            # Replace the original layer
            setattr(module, name, lora_layer)
            if verbose:
                print(f"Replaced {name} with LoRA")
        else:
            # Recursively apply to child modules
            apply_lora(child, r, alpha, verbose)

In [34]:
apply_lora(model, r=8, alpha=16, verbose=True)

Replaced lin1 with LoRA
Replaced lin2 with LoRA
Replaced lin3 with LoRA


In [35]:
# Freeze all parameters except LoRA
for param in model.parameters():
    param.requires_grad = False

for name, param in model.named_parameters():
    if "lora_" in name:
        param.requires_grad = True

In [40]:
lora_path = "checkpoints/smartlab_lora_weights_x3_45_20250416.pth"

model.load_state_dict(torch.load(lora_path), strict=False)

_IncompatibleKeys(missing_keys=['sa1_module.conv.local_nn.lins.0.weight', 'sa1_module.conv.local_nn.lins.0.bias', 'sa1_module.conv.local_nn.lins.1.weight', 'sa1_module.conv.local_nn.lins.1.bias', 'sa1_module.conv.local_nn.lins.2.weight', 'sa1_module.conv.local_nn.lins.2.bias', 'sa1_module.conv.local_nn.norms.0.module.weight', 'sa1_module.conv.local_nn.norms.0.module.bias', 'sa1_module.conv.local_nn.norms.0.module.running_mean', 'sa1_module.conv.local_nn.norms.0.module.running_var', 'sa1_module.conv.local_nn.norms.1.module.weight', 'sa1_module.conv.local_nn.norms.1.module.bias', 'sa1_module.conv.local_nn.norms.1.module.running_mean', 'sa1_module.conv.local_nn.norms.1.module.running_var', 'sa2_module.conv.local_nn.lins.0.weight', 'sa2_module.conv.local_nn.lins.0.bias', 'sa2_module.conv.local_nn.lins.1.weight', 'sa2_module.conv.local_nn.lins.1.bias', 'sa2_module.conv.local_nn.lins.2.weight', 'sa2_module.conv.local_nn.lins.2.bias', 'sa2_module.conv.local_nn.norms.0.module.weight', 'sa2_mod

In [36]:
trainable_params = [n for n, p in model.named_parameters() if p.requires_grad]
print("Trainable parameters:", trainable_params)

Trainable parameters: ['lin1.lora_A', 'lin1.lora_B', 'lin2.lora_A', 'lin2.lora_B', 'lin3.lora_A', 'lin3.lora_B']


In [27]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

pretrained_path = "checkpoints/pointnet2_s3dis_colorless_seg_x3_45_checkpoint.pth"
model = PyGPointNet2NoColorLoRa(num_classes=13, lora_r=8, lora_alpha=16) 

checkpoint = torch.load(pretrained_path)
model_state_dict = checkpoint['model_state_dict']
model.load_state_dict(model_state_dict, strict=False)




_IncompatibleKeys(missing_keys=['sa1_module.conv.local_nn.0.weight', 'sa1_module.conv.local_nn.0.bias', 'sa1_module.conv.local_nn.0.lora_A', 'sa1_module.conv.local_nn.0.lora_B', 'sa1_module.conv.local_nn.2.weight', 'sa1_module.conv.local_nn.2.bias', 'sa1_module.conv.local_nn.2.lora_A', 'sa1_module.conv.local_nn.2.lora_B', 'sa1_module.conv.local_nn.4.weight', 'sa1_module.conv.local_nn.4.bias', 'sa1_module.conv.local_nn.4.lora_A', 'sa1_module.conv.local_nn.4.lora_B', 'sa2_module.conv.local_nn.0.weight', 'sa2_module.conv.local_nn.0.bias', 'sa2_module.conv.local_nn.0.lora_A', 'sa2_module.conv.local_nn.0.lora_B', 'sa2_module.conv.local_nn.2.weight', 'sa2_module.conv.local_nn.2.bias', 'sa2_module.conv.local_nn.2.lora_A', 'sa2_module.conv.local_nn.2.lora_B', 'sa2_module.conv.local_nn.4.weight', 'sa2_module.conv.local_nn.4.bias', 'sa2_module.conv.local_nn.4.lora_A', 'sa2_module.conv.local_nn.4.lora_B', 'sa3_module.nn.0.weight', 'sa3_module.nn.0.bias', 'sa3_module.nn.0.lora_A', 'sa3_module.nn.0

In [28]:
lora_path = "checkpoints/smartlab_lora_weights_x3_45_20250416.pth"


lora_weights = torch.load(lora_path, map_location=device)


# Load LoRA weights
#lora_weights = torch.load(lora_path)
model.load_state_dict(lora_weights, strict=False)

#print(checkpoint.keys())
#print(lora_weights.keys())

#print(checkpoint['model_state_dict'])

_IncompatibleKeys(missing_keys=['sa1_module.conv.local_nn.0.weight', 'sa1_module.conv.local_nn.0.bias', 'sa1_module.conv.local_nn.2.weight', 'sa1_module.conv.local_nn.2.bias', 'sa1_module.conv.local_nn.4.weight', 'sa1_module.conv.local_nn.4.bias', 'sa2_module.conv.local_nn.0.weight', 'sa2_module.conv.local_nn.0.bias', 'sa2_module.conv.local_nn.2.weight', 'sa2_module.conv.local_nn.2.bias', 'sa2_module.conv.local_nn.4.weight', 'sa2_module.conv.local_nn.4.bias', 'sa3_module.nn.0.weight', 'sa3_module.nn.0.bias', 'sa3_module.nn.2.weight', 'sa3_module.nn.2.bias', 'sa3_module.nn.4.weight', 'sa3_module.nn.4.bias', 'fp3_module.nn.0.weight', 'fp3_module.nn.0.bias', 'fp3_module.nn.2.weight', 'fp3_module.nn.2.bias', 'fp2_module.nn.0.weight', 'fp2_module.nn.0.bias', 'fp2_module.nn.2.weight', 'fp2_module.nn.2.bias', 'fp1_module.nn.0.weight', 'fp1_module.nn.0.bias', 'fp1_module.nn.2.weight', 'fp1_module.nn.2.bias', 'fp1_module.nn.4.weight', 'fp1_module.nn.4.bias', 'mlp.0.weight', 'mlp.0.bias', 'mlp.3

In [4]:
# Pcd files
#pcd_path = "C:/Users/yanpe/OneDrive - Metropolia Ammattikorkeakoulu Oy/Research/data/smartlab/Smartlab-2024-04-05_10-58-26_colour_cleaned.pcd"
pcd_path = "C:/Users/yanpe/OneDrive - Metropolia Ammattikorkeakoulu Oy/Research/data/smartlab/SmartLab_2024_E57_Single_5mm.pcd"

pcd = o3d.io.read_point_cloud(pcd_path)

In [5]:
# Move the point cloud to its min(x,y,z) corner
 
def move_to_corner(points):    
    # Find the minimum x, y, z
    min_xyz = points.min(axis=0)
    # Translate the point cloud so that the min corner becomes the origin
    moved_points = points - min_xyz
    
    return moved_points

moved_points = move_to_corner(np.array(pcd.points))
pcd.points = o3d.utility.Vector3dVector(moved_points)

In [6]:
# Downsample the point cloud with a voxel of 0.03
downpcd = pcd.voxel_down_sample(voxel_size=0.03)

print(len(downpcd.points))

866900


In [7]:
def normalize_points_corner(points):
    # Step 1: Shift points so that the minimum x, y, z becomes the origin.
    min_vals = np.min(points, axis=0)
    shifted_points = points - min_vals  # Now the lower bound is (0,0,0)
    
    # Step 2: Compute the scaling factors from the shifted points.
    # The maximum after shifting represents the range in each dimension.
    max_vals = np.max(shifted_points, axis=0)
    scale = max_vals.copy()
    
    # Avoid division by zero if any dimension is flat.
    scale[scale == 0] = 1
    
    # Normalize the shifted points to the [0, 1] interval.
    normalized_points = shifted_points / scale

    return normalized_points

normalized = normalize_points_corner(np.array(downpcd.points))

In [8]:
# Extract coordinates and colors from the point cloud
down_points = torch.tensor(np.array(downpcd.points), dtype=torch.float32)  
down_colors = torch.tensor(np.array(downpcd.colors), dtype=torch.float32)
down_normalized = torch.tensor(normalized, dtype=torch.float32)

In [9]:
# Create a Data object with x (3 features) and pos (coordinates)
data = Data(x=down_normalized, pos=down_points)

data = data.to(device)

In [10]:
# If you have only one point cloud
dataset = [data]  # List of Data objects

num_workers = 0
batch_size = 32

# Create a DataLoader (batch_size can be adjusted as needed)
custom_loader = DataLoader(dataset, batch_size=batch_size, shuffle=False,
                         num_workers=num_workers, pin_memory=True)

In [20]:
model.eval()

import torch.profiler

with torch.profiler.profile(
    activities=[torch.profiler.ProfilerActivity.CPU, torch.profiler.ProfilerActivity.CUDA],
    record_shapes=True,
) as prof:

    with torch.no_grad():
        start_time = time.time()
        for data in custom_loader:
            data = data.to(device)
            with torch.amp.autocast("cuda"):
                predictions = model(data)
            labels = predictions.argmax(dim=-1)
            # Process the labels as needed
            #labels_arr = labels.cpu().numpy()
            # Count occurrences of labels
            unique_labels, label_counts = torch.unique(labels, return_counts=True)
            # Combine and print
            #result_labels = np.array(list(zip(unique_labels, label_counts)))
            #print("Label counts:")
            #for label, count in zip(unique_labels, label_counts):
            #    print(f"Label {label.item()}: {count.item()}")
            result_labels = torch.stack((unique_labels, label_counts), dim=1).cpu()
            print("Label counts:")
            print(result_labels)
        end_time = time.time()
        print(f"Total inference time: {end_time - start_time:.4f} seconds")  
    
print(prof.key_averages().table(sort_by="cuda_time_total"))

RuntimeError: cannot pin 'torch.cuda.FloatTensor' only dense CPU tensors can be pinned

In [ ]:
# Assuming `labels` is the tensor containing predicted labels for each point
predicted_colors = color_map[labels.cpu().numpy()]  # Shape: [num_points, 3]

# Assuming `pcd` is your Open3D point cloud object
downpcd.colors = o3d.utility.Vector3dVector(predicted_colors)

In [ ]:
# Visualize the point cloud with colored labels
o3d.visualization.draw_geometries([downpcd])

In [ ]:
# Save the point cloud to a file
save_path = "C:/Users/yanpe/OneDrive - Metropolia Ammattikorkeakoulu Oy/Research/data/smartlab/labelled/Smartlab_aalto_pcd_lora_label_pointnet2_x3_0.03_20250417.ply"
o3d.io.write_point_cloud(save_path, downpcd)
